<a href="https://colab.research.google.com/github/dapopov-st/RagOverArXivWithFinetune/blob/main/RAG_over_ArXiv_PDFs_Part5_Mistral_7b_Instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I'm using [Chris Alexiuk's](https://www.linkedin.com/in/csalexiuk/) [notebook](https://colab.research.google.com/drive/172uMprWwUfEecXQWBrsgDAlkpT_EK39z?usp=sharing)
as a starting point and plan to experiment with some of the ideas from [A Practical Approach to Retrieval Augmented Generation Systems](https://angelinamagr.gumroad.com/l/practical-approach-to-RAG-systems) by Allahyari and Yang.

## Steps
- Experiment with Mistral 7B.  May reduce hallucinations in prompt responses and be faster at inference than Zephyr 7B, which is critical. On the other hand, may require more data to finetune later since it's already aligned with synthetic datasets (or may not if use it on nonsynthetic ArXiv!)
- Also, Zephyr's data included openbmb/UltraFeedback, which included some ArXiv papers, so it may finetune better.
- Be careful if finetune on abstracts only as that may not be representative unless initial data RAGs over abstract as well.  Something to test, though.
- CONCLUSION: For task at hand, Mistral-7B-Instruct is much faster due to grouped-query attention (GQA) and sliding window attention (SWA).  Subjectively, I find that the responses are at least as good as Zephyr's and have not spotted an extreme hallucination problem.  Proceding with Mistral-7B-Instruct for now.

## Get the data and build a Retriever

- Original NB worked in under 10GB on V100

In [ ]:
!pip install -U -q "langchain" "transformers==4.35.0" "datasets==2.12.0" "tokenizers==0.14.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" "arxiv==1.4"
!pip install -U -q cohere llama-index
!pip install PyPDF2
!pip install pypdf
!pip install -q qdrant-client
!pip install -q -U faiss-cpu tiktoken sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
import transformers, datasets, tokenizers
transformers.__version__, datasets.__version__, tokenizers.__version__

('4.35.0', '2.12.0', '0.14.0')

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')

output_dir = '/content/drive/MyDrive/PdfRag/rag_output_dir'
logging_dir = '/content/drive/MyDrive/PdfRag/rag_logging_dir'
index_dir = '/content/drive/MyDrive/PdfRag/rag_index_dir'

#!ls /content/drive/MyDrive/PdfRag/clusterofstars
%cd /content/drive/MyDrive/PdfRag
#My\ Drive/PdfRag && ls clusterofstars
!ls .

Mounted at /content/drive/
/content/drive/MyDrive/PdfRag
 barbie.csv									 instructionmining
 cache										 oppenheimer.csv
 clusterofstars									 rag_index_dir
'Efficient Parallelization Layouts for Large-Scale Distributed Model Training'	 testabstract


- The documents consist of a few dozen ArXiv papers about modern LLMs

In [ ]:
from pathlib import Path
PDFS_PATH = Path('/content/drive/MyDrive/PdfRag/clusterofstars')
PDFS = list(PDFS_PATH.glob('*.pdf'))
PDFS[0], len(PDFS)

(PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/In-Context Retrieval-Augmented Language Models.pdf'),
 26)

In [ ]:
PDFS

[PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/In-Context Retrieval-Augmented Language Models.pdf'),
 PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/Toolformer: Language Models Can Teach Themselves to Use Tools.pdf'),
 PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/Skeleton-of-Thought: Large Language Models Can Do Parallel Decoding.pdf'),
 PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/SequenceMatch: Imitation Learning for Autoregressive Sequence Modelling with Backtracking.pdf'),
 PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/Chain-of-Thought Prompting Elicits Reasoning in Large Language Models.pdf'),
 PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/Training language models to follow instructions with human feedback.pdf'),
 PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/LoRA: Low-Rank Adaptation of Large Language Models.pdf'),
 PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/QLORA: Efficient Finetuning of Quantize

In [ ]:
# fastai function to clean GPU memory
import sys,gc,traceback
import torch
def clean_ipython_hist():
    # Code in this function mainly copied from IPython source
    if not 'get_ipython' in globals(): return
    ip = get_ipython()
    user_ns = ip.user_ns
    ip.displayhook.flush()
    pc = ip.displayhook.prompt_count + 1
    for n in range(1, pc): user_ns.pop('_i'+repr(n),None)
    user_ns.update(dict(_i='',_ii='',_iii=''))
    hm = ip.history_manager
    hm.input_hist_parsed[:] = [''] * pc
    hm.input_hist_raw[:] = [''] * pc
    hm._i = hm._ii = hm._iii = hm._i00 =  ''



def clean_tb():
    # h/t Piotr Czapla
    if hasattr(sys, 'last_traceback'):
        traceback.clear_frames(sys.last_traceback)
        delattr(sys, 'last_traceback')
    if hasattr(sys, 'last_type'): delattr(sys, 'last_type')
    if hasattr(sys, 'last_value'): delattr(sys, 'last_value')

def clean_mem():
    clean_tb()
    clean_ipython_hist()
    gc.collect()
    torch.cuda.empty_cache()



### Task 1: Prepare the data and  build a PDF Data Loader

In [ ]:
from PyPDF2 import PdfReader
reader = PdfReader(os.path.expanduser(PDFS[0]))
pages = reader.pages
documents = []
for page in pages:
  documents.append(page.extract_text())
#print(documents[-1])

#### First drop everything from References onwards. References were 'confusing' RAG into retrieving primarily titles of papers mentioned there, which is likely not very useful

In [ ]:
import PyPDF2

def load_pdf_to_string(pdf_path):
    # Open the PDF file in binary mode
    with open(pdf_path, 'rb') as file:
        # Create a PDF file reader object
        pdf_reader = PyPDF2.PdfReader(file)

        # Initialize an empty string to hold the text
        text = ''

        # Loop through each page and extract the text
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            page_text = page.extract_text()
            references_index= page_text.upper().find('\nREFERENCES\n')
            if references_index != -1:
              page_text = page_text[:references_index]
              text += page_text
              return text
            text += page_text
    return text

# Use the function to load a PDF into a string
text = load_pdf_to_string(os.path.expanduser(PDFS[1]))

In [ ]:
def get_title(pdf_path): return os.path.expanduser(pdf_path).split('/')[-1]

In [ ]:
get_title(PDFS[-1])

'TeacherLM: Teaching to Fish Rather Than Giving the Fish, Language Modeling Likewise.pdf'

In [ ]:
text.find('References\n')

-1

In [ ]:
PDFS[0]

PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/In-Context Retrieval-Augmented Language Models.pdf')

In [ ]:
all_docs_and_titles = [(load_pdf_to_string(os.path.expanduser(pdf_path)),get_title(pdf_path)) for pdf_path in PDFS]

In [ ]:
all_docs = [doc[0] for doc in all_docs_and_titles]
all_titles = [doc[1] for doc in all_docs_and_titles]

In [ ]:
from langchain.document_loaders.onedrive_file import CHUNK_SIZE
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter, RecursiveCharacterTextSplitter
from langchain.text_splitter import Document

CHUNK_SIZE = 1000
CHUNK_OVERLAP = 30

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap = CHUNK_OVERLAP,
    length_function=len,
)
#text_splitter.split_text(all_pages[0])
# docs = [Document(page_content=pages) for pages in all_pages]
docs  = [text_splitter.split_text(doc) for doc in all_docs]
# # docs

In [ ]:
len(docs)

26

In [ ]:
tot_len = 0
for text in docs[0]:
    tot_len += len(text)
tot_len #OK, makes sense

37667

In [ ]:
len(docs[0])

39

### Task 2: Create an "Index"

- Not yet sure if should use Qdrant or FAISS


#### Selecting the VectorStore


In [ ]:
from langchain.vectorstores import Qdrant, FAISS

In [ ]:
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)


In [ ]:
#vector_store = FAISS.from_documents(docs, embedder)
#vector_store = FAISS.from_documents((docs[i][j] for i in range(len(docs)) for j in range(len(docs[i]))), embedder)
#vector_store = FAISS.from_documents(docs, embedder)
#vector_store = FAISS.from_documents(docs[0], embedder)
from langchain.schema.document import Document

docs = [Document(page_content=doc[i],metadata={'source':all_titles[j]}) for j,doc in enumerate(docs) for i in range(len(doc))]
for index, pdf in enumerate(docs):
   content = docs[index]
   if index == 0:
       vector_store = FAISS.from_documents([content], embedder)
   else:
      vector_store_i = FAISS.from_documents([content], embedder)
      vector_store.merge_from(vector_store_i)

vector_store
#vector_store.save_local(index_dir)

In [ ]:
vector_store.save_local(index_dir)

### To reload the embeddings made above on the next Colab nb use, run the code below.

In [ ]:
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
store = LocalFileStore("./cache/")

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)
embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)

vector_store = FAISS.load_local(index_dir, embedder)

Check that the VectorStore is working by embedding a query and retrieving passages from our reviews that are close to it.

In [ ]:
query = "What is Retrieval-augmented generation?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)

1 I NTRODUCTION
State-of-the-art LLMs continue to struggle with factual errors (Mallen et al., 2023; Min et al., 2023)
despite their increased model and data scale (Ouyang et al., 2022). Retrieval-Augmented Generation
(RAG) methods (Figure 1 left; Lewis et al. 2020; Guu et al. 2020) augment the input of LLMs
with relevant retrieved passages, reducing factual errors in knowledge-intensive tasks (Ram et al.,
2023; Asai et al., 2023a). However, these methods may hinder the versatility of LLMs or introduce
unnecessary or off-topic passages that lead to low-quality generations (Shi et al., 2023) since they
retrieve passages indiscriminately regardless of whether the factual grounding is helpful. Moreover,
the output is not guaranteed to be consistent with retrieved relevant passages (Gao et al., 2023) since
the models are not explicitly trained to leverage and follow facts from provided passages. This
work introduces Self-Reflective Retrieval-augmented Generation ( SELF-RAG)to improve an
LL

In [ ]:
query = "What is Self-Rag?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)

predictions are mostly aligned with their assessments. Appendix Table 6 shows several annotated
examples and explanations on assessments.
6 C ONCLUSION
This work introduces SELF-RAG, a new framework to enhance the quality and factuality of LLMs
through retrieval on demand and self-reflection. SELF-RAGtrains an LM to learn to retrieve, generate,
and critique text passages and its own generation by predicting the next tokens from its original
vocabulary as well as newly added special tokens, called reflection tokens. SELF-RAGfurther enables
the tailoring of LM behaviors at test time by leveraging reflection tokens. Our holistic evaluations on
six tasks using multiple metrics demonstrate that SELF-RAGsignificantly outperforms LLMs with
more parameters or with conventional retrieval-augmented generation approaches.
10Preprint.
ETHICAL CONCERNS
This work aims to improve the factuality of LLM outputs, the lack of which continues to cause nu-
3 S ELF-RAG: LEARNING TO RETRIEVE , GENERATE AND C

Let's see how much time the `CacheBackedEmbeddings` pattern saves us:

In [ ]:
%%timeit -n 1 -r 1
query = "What is Self-Rag?"
embedding_vector = embedder.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

11.7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%timeit
query = "What is Self-Rag?"
embedding_vector = embedder.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

7.37 ms ± 223 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


As we can see, even over a significant number of runs - the cached query is significantly faster than the first instance of the query!

With that, we're ready to move onto Task 3!

### Task 3: Building a Retrieval Chain

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

We will be leveraging Tim Dettmer's `bitsandbytes` as well as `accelerate` and `transformers` from Hugging Face to make our model as small as possible. The overall quality of the model is fairly well retained!

In [ ]:
transformers.__version__

'4.35.0'

In [ ]:
import torch
import transformers
# BitsAndBytes for 4-bit quantization with NF4-type configuration to load  model in 4-bit precision.
# Will help load the model faster and reduce the memory footprint so that it can be run on Google Colab.
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
bng_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_id
)


model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bng_config, #using bnb_config
    device_map='auto'
)

model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )

In [ ]:
# Load the corresponding tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id
)

Now we need to pack it into a `pipeline` for compatability with `langchain`!

In [ ]:
 #contains highly optimized components some of which are not yet available in PyTorch
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 4.6 MB/s eta 0:00:00


In [ ]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=True,
    temperature=0.0,
    max_new_tokens=256
) # Get a cudann warning, likely since using T4 vs, say A100

In [ ]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

Now we can set up our chain.

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
from langchain.chains import RetrievalQA,RetrievalQAWithSourcesChain
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()

qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    callbacks=[handler],
    return_source_documents=True
)
# qa_with_sources_chain = RetrievalQAWithSourcesChain.from_chain_type(
#     llm=llm,
#     retriever=retriever,
#     callbacks=[handler],
#     return_source_documents=True
# )

- Try using RetrievalQAWithSourcesChain

In [ ]:
#qa_with_sources_chain({"question" : "What makes Self-Rag different from Rag?"})

- Well, so far my use of RetrievalQAWithSourcesChain does not seem to be working well at all!!!  Go back to RetrievalQA

In [ ]:
qa_with_sources_chain({"query" : "How does Self-Rag compare to Rag?"})

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'How does Self-Rag compare to Rag?',
 'result': ' Self-Rag is a framework that enhances the quality and factuality of LLMs through retrieval and self-reflection, while RAG is a retrieval-augmented generation approach that retrieves passages indiscriminately.',
 'source_documents': [Document(page_content='performance. In particular, we randomly sample 5k, 10k, 20k, and 50k instances from our original\n150k training instances, and fine-tune four SELF-RAG 7Bvariants on those subsets. Then, we compare\nthe model performance on PopQA, PubHealth, and ASQA (citation precision) with our final SELF-\nRAGtrained on the full 150k instances. We also evaluate Figures 4a, 4b and 4c shows the models’\nperformance trained on different amount of data. Across all datasets, increasing data size often shows\nupward trajectories and the improvements are significantly larger in PopQA and ASQA, while we do\nnot observed such significant improvements on Llama2-FT 7Bwhen increasing the training data 

- However, adding 'source' metadata to Document yielded a result that has has article title, which was one of the desired results.  If end up needing filtering articles by metadata with FAISS, see https://github.com/langchain-ai/langchain/discussions/10983.

In [ ]:
qa_with_sources_chain({"query" : "What is QLoRa?"})

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'What is QLoRa?',
 'result': ' QLoRa is a technique for reducing the memory requirements of deep learning models during training. It does this by using a combination of quantization and pruning to reduce the number of parameters in the model. QLoRa also uses a technique called LoRA, which allows the model to learn new representations for the remaining parameters. This allows the model to continue to improve its performance even with fewer parameters.',
 'source_documents': [Document(page_content='and compare QLoRA with 16-bit adapter-finetuning and with full-finetuning for models up to 3B. Our\nevaluations include GLUE [ 58] with RoBERTa-large [ 38], Super-NaturalInstructions (TKInstruct)\n[61] with T5 [ 49], and 5-shot MMLU [ 24] after finetuning LLaMA on Flan v2 [ 39] and Alpaca\n[55]. To additionally study the advantages of NF4 over other 4-bit data types, we use the setup of\nDettmers and Zettlemoyer [13] and measure post-quantization zero-shot accuracy and perplexity\nac

In [ ]:
qa_with_sources_chain({"query" : "Did these papers explore themes of existentialism?"})

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Did these papers explore themes of existentialism?',
 'result': ' No, these papers did not explore themes of existentialism.',
 'source_documents': [Document(page_content='The correctness of the augmented content is not the only\nimportant factor. The relevance of the content and the con-sistency of the reasoning logic are also significant (Wang\net al., 2022a), which allows the student models to learn to\nthink critically and truly enhance the model’s generalization\nability on unseen tasks.\nIn comparison to human annotation and text-davinci-\n003, what are the characteristics and deficiencies of the\ngeneration content of TeacherLM-7.1B?TeacherLM: Teaching to Fish Rather Than Giving the Fish, Language Modeling Likewise\nAs shown in Figure 4 and Appendix C, TeacherLM-7.1B’s\nexplanations are generally more comprehensive and detailed\nthan human annotations and text-davinci-003’s explanations.\nHowever, it falls behind text-davinci-003 in solving mathe-\nmatical problems, p

In [ ]:
#qa_with_sources_chain({"query" : " Open large language models (LLMs) with great performance in various tasks have significantly advanced the development of LLMs. However, they are far inferior to commercial models such as ChatGPT and GPT-4 when acting as agents to tackle complex tasks in the real world. These agent tasks employ LLMs as the central controller responsible for planning, memorization, and tool utilization, necessitating both fine-grained prompting methods and robust LLMs to achieve satisfactory performance. Though many prompting methods have been proposed to complete particular agent tasks, there is lack of research focusing on improving the agent capabilities of LLMs themselves without compromising their general abilities. In this work, we present AgentTuning, a simple and general method to enhance the agent abilities of LLMs while maintaining their general LLM capabilities. We construct AgentInstruct, a lightweight instruction-tuning dataset containing high-quality interaction trajectories. We employ a hybrid instruction-tuning strategy by combining AgentInstruct with open-source instructions from general domains. AgentTuning is used to instruction-tune the Llama 2 series, resulting in AgentLM. Our evaluations show that AgentTuning enables LLMs' agent capabilities without compromising general abilities. The AgentLM-70B is comparable to GPT-3.5-turbo on unseen agent tasks, demonstrating generalized agent capabilities. We open source the AgentInstruct and AgentLM-7B, 13B, and 70B models at https://github.com/THUDM/AgentTuning, serving open and powerful alternatives to commercial LLMs for agent tasks. "})

In [ ]:
from langchain.document_loaders import WebBaseLoader
from langchain.chains.summarize import load_summarize_chain

In [ ]:
query="Open large language models (LLMs) with great performance in various tasks have significantly advanced the development of LLMs. However, they are far inferior to commercial models such as ChatGPT and GPT-4 when acting as agents to tackle complex tasks in the real world. These agent tasks employ LLMs as the central controller responsible for planning, memorization, and tool utilization, necessitating both fine-grained prompting methods and robust LLMs to achieve satisfactory performance. Though many prompting methods have been proposed to complete particular agent tasks, there is lack of research focusing on improving the agent capabilities of LLMs themselves without compromising their general abilities. In this work, we present AgentTuning, a simple and general method to enhance the agent abilities of LLMs while maintaining their general LLM capabilities. We construct AgentInstruct, a lightweight instruction-tuning dataset containing high-quality interaction trajectories. We employ a hybrid instruction-tuning strategy by combining AgentInstruct with open-source instructions from general domains. AgentTuning is used to instruction-tune the Llama 2 series, resulting in AgentLM. Our evaluations show that AgentTuning enables LLMs' agent capabilities without compromising general abilities. The AgentLM-70B is comparable to GPT-3.5-turbo on unseen agent tasks, demonstrating generalized agent capabilities. We open source the AgentInstruct and AgentLM-7B, 13B, and 70B models at https://github.com/THUDM/AgentTuning, serving open and powerful alternatives to commercial LLMs for agent tasks. "

In [ ]:
os.listdir('.')

['clusterofstars',
 'cache',
 'rag_index_dir',
 'oppenheimer.csv',
 'barbie.csv',
 'testabstract',
 'instructionmining',
 'Efficient Parallelization Layouts for Large-Scale Distributed Model Training']

In [ ]:
chain = load_summarize_chain(llm, chain_type="stuff")
from langchain.document_loaders import TextLoader
query = TextLoader('testabstract').load()
#query = query_loader.load({"text" : "How does Self-Rag compare to Rag?"})
query="Which papers are most similar to the article with the following summary?  Article summary: " + chain.run(query)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
qa_with_sources_chain({"query" : query})

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.


{'query': "Which papers are most similar to the article with the following summary?  Article summary: \n\nThe paper presents AgentTuning, a method to enhance the agent capabilities of large language models (LLMs) while maintaining their general LLM capabilities. The authors construct AgentInstruct, a lightweight instruction-tuning dataset, and use a hybrid instruction-tuning strategy to instruction-tune the Llama 2 series, resulting in AgentLM. The evaluations show that AgentTuning enables LLMs' agent capabilities without compromising general abilities. The AgentLM-70B is comparable to GPT-3.5-turbo on unseen agent tasks, demonstrating generalized agent capabilities. The paper opens source the AgentInstruct and AgentLM-7B, 13B, and 70B models.",
 'result': ' \n\nThe papers most similar to the article are: \n\n1. "AgentTuning: Enhancing Agent Capabilities of Large Language Models" by Yoshua Bengio, et al. \n2. "AgentLM: Instruction-Tuning Large Language Models" by Yoshua Bengio, et al. 

In [ ]:
chain = load_summarize_chain(llm, chain_type="stuff")
from langchain.document_loaders import TextLoader
query = TextLoader('instructionmining').load()
#query = query_loader.load({"text" : "How does Self-Rag compare to Rag?"})
query="Which papers are most similar to the article with the following summary?  Article summary: " + chain.run(query)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
qa_with_sources_chain({"query" : query})

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Which papers are most similar to the article with the following summary?  Article summary: \n\nThe paper proposes InstructMining, a method for automatically selecting high-quality instruction-following data for finetuning large language models (LLMs). The method uses natural language indicators to evaluate unseen datasets and applies double descent phenomenon and BlendSearch to optimize the process. Experiment results show that InstructMining achieves state-of-the-art performance on two benchmarks.',
 'result': ' The papers most similar to the article with the summary are:\n\n1. "Automatic Selection of High-Quality Instruction-Following Data for Fine-tuning Large Language Models" by Khashabi et al. (2020)\n2. "InstructMining: A Method for Automatically Selecting High-Quality Instruction-Following Data for Fine-tuning Large Language Models" by Khashabi et al. (2021)\n\n1. "Automatic Selection of High-Quality Instruction-Following Data for Fine-tuning Large Language Models" by

In [ ]:
chain = load_summarize_chain(llm, chain_type="stuff")
from langchain.document_loaders import TextLoader
query = TextLoader('instructionmining').load()
#query = query_loader.load({"text" : "How does Self-Rag compare to Rag?"})
query="Which papers are most different to the article with the following summary?  Article summary: " + chain.run(query)
qa_with_sources_chain({"query" : query})

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'query': 'Which papers are most different to the article with the following summary?  Article summary: \n\nThis paper proposes a new method called InstructMining for selecting high-quality instruction-following data to optimize the performance of large language models (LLMs) in interacting with humans. The method uses natural language indicators to evaluate the quality of unseen datasets and leverages BlendSearch to find the best subset of the entire dataset. Experiment results show that InstructMining achieves state-of-the-art performance on two popular benchmarks. Additionally, the paper discovers the double descent phenomenon in LLM finetuning and provides insights into the relationship between dataset size and model performance.',
 'result': '\n\nThe papers that are most different to the article with the given summary are:\n\n1. [28] E. J. Hu, Y. Shen, P. Wallis, Z. Allen-Zhu, Y. Li, S. Wang, L. Wang, and W. Chen. Lora: Low-rank adaptation of large language models. arXiv preprint 

### ArXiv API

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
locale.getpreferredencoding() ### SOLVED THE UTF-8 ISSUE!

'UTF-8'

In [ ]:
!pip install arxiv

In [ ]:
#from langchain.retrievers import ArxivRetrieverM # GETTING ERROR, probably updated API
from langchain.retrievers.arxiv import ArxivRetriever

In [ ]:
retriever = ArxivRetriever(load_max_docs=2)

In [ ]:
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 47.9 MB/s eta 0:00:00


In [ ]:
docs = retriever.get_relevant_documents(query="2311.05610")

In [ ]:
docs

[Document(page_content='Efficiently training large language models requires parallelizing across\nhundreds of hardware accelerators and invoking various compute and memory\noptimizations. When combined, many of these strategies have complex\ninteractions regarding the final training efficiency. Prior work tackling this\nproblem did not have access to the latest set of optimizations, such as\nFlashAttention or sequence parallelism. In this work, we conduct a\ncomprehensive ablation study of possible training configurations for large\nlanguage models. We distill this large study into several key recommendations\nfor the most efficient training. For instance, we find that using a micro-batch\nsize of 1 usually enables the most efficient training layouts. Larger\nmicro-batch sizes necessitate activation checkpointing or higher degrees of\nmodel parallelism and also lead to larger pipeline bubbles. Our most efficient\nconfigurations enable us to achieve state-of-the-art training efficiency\

In [ ]:
docs[0].page_content

'Efficiently training large language models requires parallelizing across\nhundreds of hardware accelerators and invoking various compute and memory\noptimizations. When combined, many of these strategies have complex\ninteractions regarding the final training efficiency. Prior work tackling this\nproblem did not have access to the latest set of optimizations, such as\nFlashAttention or sequence parallelism. In this work, we conduct a\ncomprehensive ablation study of possible training configurations for large\nlanguage models. We distill this large study into several key recommendations\nfor the most efficient training. For instance, we find that using a micro-batch\nsize of 1 usually enables the most efficient training layouts. Larger\nmicro-batch sizes necessitate activation checkpointing or higher degrees of\nmodel parallelism and also lead to larger pipeline bubbles. Our most efficient\nconfigurations enable us to achieve state-of-the-art training efficiency\nresults over a range o

In [ ]:
title, query = docs[0].metadata['Title'],docs[0].page_content.replace('\n', ' ')

In [ ]:
query="Be thorough and explain your reasoning step by step.  Which of the retrieved documents has the most in common with the article with the following summary?  .  Article summary: " + query
qa_with_sources_chain({"query" : query})

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Be thorough and explain your reasoning step by step.  Which of the retrieved documents has the most in common with the article with the following summary?  .  Article summary: Efficiently training large language models requires parallelizing across hundreds of hardware accelerators and invoking various compute and memory optimizations. When combined, many of these strategies have complex interactions regarding the final training efficiency. Prior work tackling this problem did not have access to the latest set of optimizations, such as FlashAttention or sequence parallelism. In this work, we conduct a comprehensive ablation study of possible training configurations for large language models. We distill this large study into several key recommendations for the most efficient training. For instance, we find that using a micro-batch size of 1 usually enables the most efficient training layouts. Larger micro-batch sizes necessitate activation checkpointing or higher degrees of m

In [ ]:
clean_mem()